In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**아래 text에 원하는 리뷰 데이터 텍스트를 가져다가 넣으면 됨**

Big, rich and off-dry, this is powered by intense spiciness and rounded texture. Lychees dominate the fruit profile, giving an opulent feel to the aftertaste. Drink now.

In [17]:
review = "Big, rich and off-dry, this is powered by intense spiciness and rounded texture. Lychees dominate the fruit profile, giving an opulent feel to the aftertaste. Drink now."

**전처리 단계**

In [18]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
swlist = stopwords.words('english')

def get_lemma(sentence):
  lemmas = []
  for word in sentence:
    words = lemmatizer.lemmatize(word, 'v')
    lemmas.append(words)

  return lemmas

def preprocessing(sentence):
  sentence = re.sub('[^a-zA-Z]', ' ', sentence)
  sentence = sentence.lower()
  sentence = [word for word in sentence.split() if word not in swlist]
  sentence = get_lemma(sentence)

  return ' '.join(sentence)

review_preprocessed = pd.Series(preprocessing(review))

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

import json

with open('/content/drive/MyDrive/AIB/Section4/wordIndex.json') as json_file:
  word_index = json.load(json_file)
  tokenizer.word_index = word_index

# vocab_size = len(tokenizer.word_index)

review_encoded = tokenizer.texts_to_sequences(review_preprocessed)

from keras.preprocessing.sequence import pad_sequences

maxlen = 50

review_train = pad_sequences(review_encoded, maxlen = maxlen, truncating = 'post', padding = 'post')

**모델을 통하여 해당 와인 리뷰 텍스트를 바탕으로 와인을 긍정적으로 평가했는지 부정적으로 평가했는지 판정**

In [19]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/AIB/Section4/pj4model.h5')

prediction = model.predict(review_train)

prediction

if prediction >= 0.5 :
  print('이 와인을 긍정적으로 평가했습니다!')
else :
  print('이 와인을 부정적으로 평가했습니다.')

이 와인을 긍정적으로 평가했습니다!
